# How to Prepare data for ZRP Predictions
The purpose of this notebook is to illustrate how to use `ZRP_Prepare`, a module that prepares user input data for generating predictions, models, & analysis. 

In [2]:
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi=False

In [3]:
from os.path import join, expanduser, dirname
import pandas as pd
import sys
import os
import re
import warnings

In [ ]:
warnings.filterwarnings(action='ignore')
home = expanduser('~')

src_path = '{}/zrp'.format(home)
sys.path.append(src_path)

In [5]:
from zrp.prepare.prepare import ZRP_Prepare
from zrp.prepare.utils import load_file

/home/kam/.conda/envs/zrp_q1_22/lib/python3.7/site-packages/statsmodels/compat/pandas.py:35: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  version = LooseVersion(pd.__version__)
/home/kam/.conda/envs/zrp_q1_22/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


## Load sample data for prediction
Load processed list of New Jersey Mayors downloaded from https://www.nj.gov/dca/home/2022mayors.csv 

In [6]:
nj_mayors = load_file("../2022-nj-mayors-sample.csv")
nj_mayors.shape

(462, 9)

In [7]:
nj_mayors

,first_name,middle_name,last_name,house_number,street_address,city,state,zip_code,ZEST_KEY
0,Gabe,NaN,Plumer,782,Frenchtown Road,Milford,NJ,08848,2
1,Ari,NaN,Bernstein,500,West Crescent Avenue,Allendale,NJ,07401,4
2,David,J.,Mclaughlin,125,Corlies Avenue,Allenhurst,NJ,07711-1049,5
3,Thomas,C.,Fritts,8,North Main Street,Allentown,NJ,08501-1607,6
4,P.,NaN,McCkelvey,49,South Greenwich Street,Alloway,NJ,08001-0425,7
...,...,...,...,...,...,...,...,...,...
457,William,NaN,Degroff,3943,Route,Chatsworth,NJ,08019,558
458,Joseph,NaN,Chukwueke,200,Cooper Avenue,Woodlynne,NJ,08107-2108,559
459,Paul,NaN,Sarlo,85,Humboldt Street,Wood-Ridge,NJ,07075-2344,560
460,Craig,NaN,Frederick,120,Village Green Drive,Woolwich Township,NJ,08085-3180,562


#### ZRP Prepare  
To prepare the data we will use `ZRP_Prepare` 

Input data into the prediction/modeling pipeline is tabluar data with the following columns: first name, middle name, last name, house number, street address (street name), city, state, zip code, and zest key. The `ZEST_KEY` must be specified to establish correspondence between inputs and outputs; it's effectively used as an index for the data table.

`ZRP_Prepare` is used to process this input data into the set of requisite feature vectors necessary for prediction. When called, the `.transform()` function's processing steps include geocoding the data (converting addresses to block groups or census tracts), and matching the geocoded data on American Community Survey data lookup tables. This ultimately links input data to additional  demographic data based on individuals' geography. In the end, the input data is bolstered with additional features, which are used for predictions with enhanced feature fidelity. 

In [13]:
%%time
prepare = ZRP_Prepare()
prepare.fit(nj_mayors)
zrp_output = prepare.transform(nj_mayors)

  0%|          | 0/462 [00:00<?, ?it/s][Parallel(n_jobs=49)]: Using backend ThreadingBackend with 49 concurrent workers.
[Parallel(n_jobs=49)]: Done 102 tasks      | elapsed:    0.0s
100%|██████████| 462/462 [00:00<00:00, 14624.23it/s]

Data is loaded
Directory already exists
   [Start] Validating input data
     Number of observations: 462
     Is key unique: True
       (Warning!!) middle_name is 68.3982683982684% missing, this may impact the ability to return race approximations
   [Completed] Validating input data

   Formatting P1
   Formatting P2
   reduce whitespace

[Start] Preparing geo data

  The following states are included in the data: ['NJ']
   ... on state: NJ

   Data is loaded
   [Start] Processing geo data
/home/kam/zrp/zrp/prepare/../data/processed
      ...address cleaning



[Parallel(n_jobs=49)]: Done 352 tasks      | elapsed:    0.0s
[Parallel(n_jobs=49)]: Done 462 out of 462 | elapsed:    0.0s finished
sys:1: ResourceWarning: unclosed socket <zmq.Socket(zmq.PUSH) at 0x7fcfedc403d0>


      ...replicating address
         ...Base
         ...Map street suffixes...
         ...Mapped & split by street suffixes...
         ...Number processing...

         Address dataframe expansion is complete! (n=900)
         ...Base
         ...Number processing...
         House number dataframe expansion is complete! (n=900)
      ...formatting
   [Completed] Processing geo data
   [Start] Mapping geo data


/home/kam/.conda/envs/zrp_q1_22/lib/python3.7/site-packages/pandas/compat/_optional.py:116: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if distutils.version.LooseVersion(version) < minimum_version:
/home/kam/.conda/envs/zrp_q1_22/lib/python3.7/site-packages/setuptools/_distutils/version.py:351: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  other = LooseVersion(other)


      ...merge user input & lookup table
      ...mapping


/home/kam/.conda/envs/zrp_q1_22/lib/python3.7/site-packages/pandas/compat/_optional.py:116: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if distutils.version.LooseVersion(version) < minimum_version:
/home/kam/.conda/envs/zrp_q1_22/lib/python3.7/site-packages/setuptools/_distutils/version.py:351: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  other = LooseVersion(other)
100%|██████████| 1/1 [00:04<00:00,  4.98s/it]

   [Completed] Validating input geo data
Directory already exists
Output saved
   [Completed] Mapping geo data

[Completed] Preparing geo data

[Start] Preparing ACS data
   [Start] Validating ACS input data
     Number of observations: 462
     Is key unique: True
       (Warning!!) middle_name is 68.3982683982684% missing, this may impact the ability to return race approximations



/home/kam/.conda/envs/zrp_q1_22/lib/python3.7/site-packages/pandas/compat/_optional.py:116: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if distutils.version.LooseVersion(version) < minimum_version:
/home/kam/.conda/envs/zrp_q1_22/lib/python3.7/site-packages/setuptools/_distutils/version.py:351: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  other = LooseVersion(other)



   [Completed] Validating ACS input data

   ...loading ACS lookup tables
   ... combining ACS & user input data
 ...Copy dataframes
 ...Block group
 ...Census tract
 ...Zip code
 ...No match
 ...Merge
 ...Merging complete
[Complete] Preparing ACS data

CPU times: user 36.3 s, sys: 14.6 s, total: 50.9 s
Wall time: 22.8 s


### Inspect the output
- Preview the data
- View what artifacts are saved

In [9]:
zrp_output.shape

(1424, 927)

In [10]:
zrp_output.head()

,B01003_001,B02001_001,B02001_002,B02001_003,B02001_004,B02001_005,B02001_006,B02001_007,B02001_008,B02001_009,...,acs_source,city,first_name,house_number,last_name,middle_name,state,street_address,zest_in_state_fips,zip_code
ZEST_KEY,,,,,,,,,,,,,,,,,,,,,
56,1382,1382,728,86,0,483,0,0,85,40,...,BG,BRIDGEWATER,MATTHEW,100,MOENCH,C,NJ,COMMONS WAY,NaN,08807
202,3250,3250,3170,66,0,14,0,0,0,0,...,BG,MULLICA HILL,LOUIS,114,MANZO,None,NJ,BRIDGETON PIKE,NaN,08062
204,469,469,430,0,0,39,0,0,0,0,...,BG,HASBROUCK HEIGHTS,JOHN,218,DELORENZO,None,NJ,BOULEVARD,NaN,07604
224,1236,1236,1021,52,44,7,0,83,29,21,...,BG,BRIDGETON,PAUL,590,RITTER,J,NJ,SHILOH PIKE,NaN,08302
248,1199,1199,1155,0,0,14,0,0,30,0,...,BG,LAUREL SPRINGS,THOMAS,135,BARBERA,None,NJ,BROADWAY,NaN,08021


`ZRP_Prepare` generates multiple artifacts that are automatically saved:
- Dataframe with address to GEOID mappings
    - `Zest_Geocoded_test_{year}__{state_fips}.parquet`
- Validation dictionary for input data
    - `input_validator.json`
- Validation dictionary for geographic data
    - `input_geo_validator.json`
- Validation dictionary for American Community Survey data
    - `input_acs_validator.json`


In [11]:
!ls artifacts

input_acs_validator.json  input_validator.json
input_geo_validator.json  Zest_Geocoded_test_2019__34.parquet


In [15]:
zrp_output.filter(regex="BG").sample(3)

,GEOID_BG
ZEST_KEY,
231,NaN
110,340230003002
445,340258121002
